# Test for TFrecords

In [ ]:
# get some file pairs and corresponding annotations
'''
filename_pairs = [('./left_images/0.png', './right_images/0.png'),
                 ('./left_images/1.png', './right_images/1.png'),
                 ('./left_images/2.png', './right_images/2.png')]
'''
mystring = '.png'
f = open('./txt_files/image_files.txt', 'rb')
left_names = f.readlines()
f.close()
left_names = [l.strip('\n\r') for l in left_names]
left_names = [s + mystring for s in mylist]

f = open('./txt_files/word_files.txt', 'rb')
right_names = f.readlines()
f.close()
right_names = [l.strip('\n\r') for l in right_names]

filename_pairs = zip(left_names, right_names)
print filename_pairs[-1]

In [ ]:
%matplotlib inline

import cv2
import numpy as np
import skimage.io as io
import tensorflow as tf

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

tfrecords_filename = 'siamese.tfrecords'
writer = tf.python_io.TFRecordWriter(tfrecords_filename)

# Let's collect the real images to later on compare
# to the reconstructed ones
original_images = []

for img_path, annotation_path in filename_pairs:
    
    left_img = cv2.imread(img_path)
    right_img = cv2.imread(annotation_path)
    
    # we have to know sizes
    # of images to later read raw serialized string,
    # convert to 1d array and convert to respective
    # shape that image used to have.
    height = left_img.shape[0]
    width = left_img.shape[1]
    
    # Put in the original images into array
    # Just for future check for correctness
    original_images.append((left_img, right_img))
    
    img_raw = left_img.tostring()
    annotation_raw = right_img.tostring()
    #label = label.tostring()
    
    example = tf.train.Example(features=tf.train.Features(feature={
        'height': _int64_feature(height),
        'width': _int64_feature(width),
        'left_image': _bytes_feature(img_raw),
        'right_image': _bytes_feature(annotation_raw)}))
    
    writer.write(example.SerializeToString())

writer.close()

In [ ]:
## this part is for constructing the images

reconstructed_images = []

record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)
print record_iterator

for string_record in record_iterator:
    
    example = tf.train.Example()
    example.ParseFromString(string_record)
    
    height = int(example.features.feature['height']
                                 .int64_list
                                 .value[0])
    
    width = int(example.features.feature['width']
                                .int64_list
                                .value[0])
    
    img_string = (example.features.feature['image_raw']
                                  .bytes_list
                                  .value[0])
    
    annotation_string = (example.features.feature['mask_raw']
                                .bytes_list
                                .value[0])
    
    img_1d = np.fromstring(img_string, dtype=np.uint8)
    reconstructed_img = img_1d.reshape((height, width, -1))
    
    annotation_1d = np.fromstring(annotation_string, dtype=np.uint8)
    
    # Annotations have depth (3rd dimension)
    reconstructed_annotation = annotation_1d.reshape((height, width, -1))
    
    reconstructed_images.append((reconstructed_img, reconstructed_annotation))

In [ ]:
# Let's check if the reconstructed images match
# the original images

print len(original_images)
print len(reconstructed_images)

for original_pair, reconstructed_pair in zip(original_images, reconstructed_images):
    
    img_pair_to_compare, annotation_pair_to_compare = zip(original_pair,
                                                          reconstructed_pair)
    #print(np.allclose(*img_pair_to_compare))
    #print(np.allclose(*annotation_pair_to_compare))

# reading the records in a tf graph

In [1]:
%matplotlib inline

import tensorflow as tf
import skimage.io as io

IMAGE_HEIGHT = 227
IMAGE_WIDTH = 227

tfrecords_filename = 'siamese_train.tfrecords'

def read_and_decode(filename_queue):
    
    reader = tf.TFRecordReader()

    _, serialized_example = reader.read(filename_queue)

    features = tf.parse_single_example(
      serialized_example,
      # Defaults are not specified since both keys are required.
      features={
        'height': tf.FixedLenFeature([], tf.int64),
        'width': tf.FixedLenFeature([], tf.int64),
        'left_image': tf.FixedLenFeature([], tf.string),
        'right_image': tf.FixedLenFeature([], tf.string),
        'label': tf.FixedLenFeature([], tf.int64)
        })

    # Convert from a scalar string tensor (whose single string has
    # length mnist.IMAGE_PIXELS) to a uint8 tensor with shape
    # [mnist.IMAGE_PIXELS].
    image = tf.decode_raw(features['left_image'], tf.uint8)
    annotation = tf.decode_raw(features['right_image'], tf.uint8)
    
    height = tf.cast(features['height'], tf.int32)
    width = tf.cast(features['width'], tf.int32)
    
    label = tf.cast(features['label'], tf.int32)
    
    image_shape = tf.stack([height, width, 3])
    annotation_shape = tf.stack([height, width, 3])
    
    image = tf.reshape(image, image_shape)
    annotation = tf.reshape(annotation, annotation_shape)
    
    image_size_const = tf.constant((IMAGE_HEIGHT, IMAGE_WIDTH, 3), dtype=tf.int32)
    annotation_size_const = tf.constant((IMAGE_HEIGHT, IMAGE_WIDTH, 3), dtype=tf.int32)
    
    # Random transformations can be put here: right before you crop images
    # to predefined size. To get more information look at the stackoverflow
    # question linked above.
    
    resized_image = tf.image.resize_image_with_crop_or_pad(image=image,
                                           target_height=IMAGE_HEIGHT,
                                           target_width=IMAGE_WIDTH)
    
    resized_annotation = tf.image.resize_image_with_crop_or_pad(image=annotation,
                                           target_height=IMAGE_HEIGHT,
                                           target_width=IMAGE_WIDTH)
    
    
    images, annotations, labels = tf.train.shuffle_batch( [resized_image, resized_annotation, label],
                                                 batch_size=2,
                                                 capacity=30,
                                                 num_threads=2,
                                                 min_after_dequeue=10)
    
    return images, annotations, labels

In [2]:
filename_queue = tf.train.string_input_producer(
    [tfrecords_filename], num_epochs=10)

# Even when reading in multiple threads, share the filename
# queue.
image, annotation, labels = read_and_decode(filename_queue)
print image.shape, annotation.shape, labels.shape

# The op for initializing the variables.
init_op = tf.group(tf.global_variables_initializer(),
                   tf.local_variables_initializer())


with tf.Session()  as sess:
    
    sess.run(init_op)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    # Let's read off 3 batches just for example
    for i in xrange(3):
    
        img, anno, _y = sess.run([image, annotation, labels])
        print(img[0, :, :, :].shape)
        
        print('current batch')
        
        

(2, 227, 227, 3) (2, 227, 227, 3) (2,)
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, Name: <unknown>, Feature: label (data type: int64) is required but could not be found.
	 [[Node: ParseSingleExample/ParseExample/ParseExample = ParseExample[Ndense=5, Nsparse=0, Tdense=[DT_INT64, DT_INT64, DT_STRING, DT_STRING, DT_INT64], dense_shapes=[[], [], [], [], []], sparse_types=[], _device="/job:localhost/replica:0/task:0/cpu:0"](ParseSingleExample/ExpandDims, ParseSingleExample/ParseExample/ParseExample/names, ParseSingleExample/ParseExample/ParseExample/dense_keys_0, ParseSingleExample/ParseExample/ParseExample/dense_keys_1, ParseSingleExample/ParseExample/ParseExample/dense_keys_2, ParseSingleExample/ParseExample/ParseExample/dense_keys_3, ParseSingleExample/ParseExample/ParseExample/dense_keys_4, ParseSingleExample/ParseExample/Const, ParseSingleExample/ParseExample/Const_1, ParseSingleExample/ParseExample/Const_2, Par

OutOfRangeError: RandomShuffleQueue '_1_shuffle_batch/random_shuffle_queue' is closed and has insufficient elements (requested 2, current size 0)
	 [[Node: shuffle_batch = QueueDequeueManyV2[component_types=[DT_UINT8, DT_UINT8, DT_INT32], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](shuffle_batch/random_shuffle_queue, shuffle_batch/n)]]

Caused by op u'shuffle_batch', defined at:
  File "/scratch/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/scratch/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/scratch/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/scratch/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/scratch/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    io